In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
client = openai.OpenAI()

## 일반 프롬프트 vs Zero-shot CoT

In [ ]:
# 일반 프롬프트
prompt = '123 x 31은 얼마인가요?'

# zero-shot CoT 적용 프롬프트
cot_prompt = '단계별로 생각해보세요: 123 x 31은 얼마인가요?'

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": prompt}]
)


cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [7]:
res.choices[0].message.content

'123 x 31은 3813입니다.'

In [ ]:
print(cot_res.choices[0].message.content)

123 x 31 계산을 단계별로 나누어 생각해보겠습니다.

1. 먼저, 123에 1을 곱합니다. (123 x 1 = 123)
2. 다음으로, 123에 30을 곱합니다. 이는 사실상 123에 3을 곱한 다음에 그 결과에 10을 곱하는 것과 같습니다. (123 x 3 = 369, 369 x 10 = 3690)
3. 이 둘을 더합니다. (123 + 3690 = 3813).

따라서, 123 x 31 = 3813입니다.


## Few-shot CoT

In [10]:
few_shot_cot = """
Q: 123 x 31 은 얼마인가요?
A: 123 x 31을 계산해 보겠습니다.

1. 123를 분해합니다:  123 = 100 + 20 + 3 
각 항을 31과 곱합니다:
( 100 x 31 = 3100 )
( 20 x 31 = 620 )
( 3 x 31 = 93 )

2. 이제 이 결과들을 더합니다:
 3100 + 620 + 93 

3. 계산을 진행하면:
 3100 + 620 = 3720 
 3720 + 93 = 3813 

따라서, ( 123 x31 = 3813 )입니다.

Q: 789 x 56 은 얼마인가요?
A: 
"""

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": few_shot_cot}]
)

In [11]:
print(res.choices[0].message.content)

789 x 56을 계산해 보겠습니다.

1. 789를 분해합니다:  789 = 700 + 80 + 9
각 항을 56과 곱합니다:
( 700 x 56 = 39200 )
( 80 x 56 = 4480 )
( 9 x 56 = 504 )

2. 이제 이 결과들을 더합니다:
 39200 + 4480 + 504 

3. 계산을 진행하면:
 39200 + 4480 = 43680 
 43680 + 504 = 44184 

따라서, ( 789 x 56 = 44184 )입니다.


## 논리 문제 해결

In [12]:
prompt = "앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리보다 나이가 많지만, 밥보다는 어리다. 프랭크(Frank)는 데이비드보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"
cot_prompt = "단계적으로 생각해서 대답해주세요: 앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리보다 나이가 많지만, 밥보다는 어리다. 프랭크(Frank)는 데이비드보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

In [13]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": prompt}]
)


cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [14]:
print(res.choices[0].message.content)

가장 어린 사람은 프랭크(Frank)입니다.


In [15]:
print(cot_res.choices[0].message.content)

먼저 앨리스, 밥, 찰리, 데이비드, 에밀리, 프랭크가 각각의 상대에 비해 어리거나 많은지를 확인해보겠습니다.

1. 앨리스는 밥보다 나이가 많습니다. 즉, 앨리스 > 밥
2. 밥은 찰리보다 나이가 많습니다. 따라서, 앨리스 > 밥 > 찰리
3. 데이비드는 앨리스보다 어립니다. 찰리, 밥, 앨리스 > 데이비드
4. 데이비드는 에밀리보다 나이가 많습니다. 따라서, 찰리, 밥, 앨리스 > 데이비드 > 에밀리
5. 찰리는 프랭크보다 나이가 많습니다. 이를 통해 찰리, 밥, 앨리스 > 데이비드, 프랭크 > 에밀리라는 사실을 알 수 있습니다.
6. 에밀리는 찰리보다 나이가 많지만 밥보다는 어립니다. 이는 에밀리 > 찰리이며, 밥 > 에밀리라는 의미입니다. 이 정보를 활용하면, 밥, 앨리스 > 데이비드 > 에밀리 > 찰리, 프랭크라는 순서를 만들 수 있습니다.
7. 마지막으로 프랭크는 데이비드보다 어립니다. 따라서, 앨리스, 밥 > 데이비드 > 에밀리 > 찰리 > 프랭크라는 순서를 얻을 수 있습니다.

따라서, 이 정보들을 종합하면 가장 어린 사람은 프랭크임을 알 수 있습니다.


## 코드 디버깅

In [16]:
prompt = """
다음과 같이 코드를 작성했더니 에러가 발생합니다. 왜죠?

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""
cot_prompt = """
다음과 같이 코드를 작성했더니 에러가 발생합니다.
실수가 무엇인지 찾아 단계별로 설명해 주세요.

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

In [17]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": prompt}]
)


cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [18]:
print(res.choices[0].message.content)

다음과 같은 코드는 인덴트 오류를 발생시킵니다. 파이썬은 공백이나 탭을 코드의 구조를 정의하는 데 중요하게 사용합니다. 여기에서 return 문은 add_numbers 함수의 본문이므로, 해당 함수의 본문에서 한 단계 들여써야 합니다. 

수정된 코드는 아래와 같습니다.

```python
def add_numbers(a, b):
  return a + b

print(add_numbers(1, 2))
```

이 코드는 두 숫자를 더하는 함수를 정의하고, 이 함수를 이용하여 1과 2를 더한 결과를 출력합니다. 이 코드는 '3'을 출력합니다.


In [19]:
print(cot_res.choices[0].message.content)

주어진 코드 영역에서 발생하는 에러는 파이썬의 들여쓰기 규칙에 위반되기 때문입니다. 위 코드는 def로 함수를 정의하는 라인 다음에 본문이 있는데, 이 본문은 함수와 동일한 라인에 작성되었습니다. 이는 파이썬의 함수 정의 규칙에 위반됩니다.

수정된 코드는 다음과 같이 작성하면 됩니다.

def add_numbers(a, b):
    return a + b

print(add_numbers(1, 2))

위와 같이 함수의 본문은 반드시 함수 정의 라인보다 들여써야 합니다. 이렇게 수정하면 '3'이라는 결과를 볼 수 있습니다.
